In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import csv
import os
import sys
sys.path.append("..")
import minimise
# Enable or disable Tensor Float 32 Execution
tf.config.experimental.enable_tensor_float_32_execution(False)
import matplotlib.pyplot as plt
import scipy.constants as const


def write_profile(filename, centers, densities_H, densities_O):
    """
    Write the density profiles to a file.

    Parameters:
    - filename (str): Output file name.
    - centers (np.ndarray): Bin centers.
    - densities_H (np.ndarray): Density values for component H.
    - densities_O (np.ndarray): Density values for component O.
    """
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=' ')
        writer.writerow(["xbins", "rho_H", "rho_O"])
        for center, density_H, density_O in zip(centers, densities_H, densities_O):
            writer.writerow([f"{center:.4f}", f"{density_H:.20f}", f"{density_O:.20f}"])


def LJ_wall_93(z_range, z_wall, epsilon, sigma, cutoff, place='lo'):
    """
    Calculate the Lennard-Jones wall potential for a range of z values.
 
    epsilon is in kT
    sigma is in Angstrom
    cutoff is in Angstrom
    """
 
    hilo = {'lo': 1.0, 'hi': -1.0}
    z_rel = hilo[place]*(z_range - z_wall)
    
    # Avoid division by zero and negative values in z_rel
    with np.errstate(divide='ignore', invalid='ignore'):
        ratio3 = np.where(z_rel > 0, (sigma / z_rel) ** 3, 0)
        ratio9 = ratio3 ** 3
        energy = epsilon * (ratio9 * 2./15. - ratio3)
        
    # Apply boundary conditions
    energy[z_rel <= 0] = np.inf
    energy[z_rel > cutoff] = 0.0
    
    # Shift the potential to ensure it is zero at the cutoff
    cutoff_energy = epsilon * ((sigma / cutoff) ** 9 * 2./15. - (sigma / cutoff) ** 3)
    energy[z_rel <= cutoff] -= cutoff_energy
    
    return energy




def do_slit(Efield, mu_H, mu_O, q_H, q_O, temp, modelH_path, modelO_path, results_path):
    """
    Determine the self-consistent density profiles 
    
    
    Parameters:
    - Efield (float): Electric field, in real lammps unit, one would use add force q*Efield
    - mu_H (float): Chemical potential for H.
    - mu_O (float): Chemical potential for O.
    - q_H (float): Charge for H.
    - q_O (float): Charge for O.
    - temp (float): Temperature in Kelvin.
    - modelH_path (str): Path to the Keras model for H.
    - modelO_path (str): Path to the Keras model for O.
    - results_path (str): Path to save the results.
    - dielectric (float): Dielectric constant.
    - kappa_inv (float): for Coulommic splitting.
    """
    model_H = keras.models.load_model(modelH_path)
    model_O = keras.models.load_model(modelO_path)
    
    
    kappa_inv = 5.0
    dielectric = 1.0
  
    os.makedirs(results_path, exist_ok=True)

    
    #output_file = f'{results_path}/slit-muH{mu_H:.3f}-muO{mu_O:.3f}-E{Efield:.1f}.out'
    #if os.path.exists(output_file):
    #        print(f"Skipping as {output_file} already exists.")
    #        return None, None, None
    
    zbins = np.arange(0, 50, 0.03)
    
    V_lo = LJ_wall_93(zbins, 5, 1.0, 1.0, 0.858374218933, place='lo')
    V_hi = LJ_wall_93(zbins, 45, 1.0, 1.0, 0.858374218933, place='hi')
    Vext = V_lo + V_hi
    
    
    conversion = const.Boltzmann * temp * const.Avogadro / 4184
    gradient = Efield / conversion
    ez = gradient * (zbins - 25)
    

    muloc_H = - Vext + q_H * ez + mu_H
    muloc_O = - Vext + q_O * ez + mu_O    

    
    
    zs, rho_H, rho_O = minimise.minimise_LR_twotype(model_H, model_O, zbins, muloc_H, muloc_O,
                                                    q_H, q_O, kappa_inv, temp, dielectric, 
                                                    initial_guess=0.01 ,input_bins=667, plot=True, 
                                                    tolerance=1e-5, tolerance_restr = 1e-3, maxiter=50000,
                                                    plot_every=200, print_every=200)
    #if zs is not None:
    #    write_profile(output_file, zs, rho_H, rho_O)
        
    return zs, rho_H, rho_O
    


## RPM model

In [ ]:

modelH_path = "../../models/RPM_H.keras"
modelO_path = "../../models/RPM_O.keras"
results_path = "../../tests/RPM_all_LR"

z_range, rho_H, rho_O = do_slit(4.5, -12, -12, 1.0, -1.0, 4000, modelH_path, modelO_path, results_path)


## PM model

In [ ]:

modelH_path = "../../models/PM_H.keras"
modelO_path = "../../models/PM_O.keras"
results_path = "../../tests/PM_all_LR"

z_range, rho_H, rho_O = do_slit(5.0, -12.149121669962774, -10.45953443451203, 1.0, -1.0, 4000, modelH_path, modelO_path, results_path)


## 2:1 PM model

In [ ]:

modelH_path = "../../models/21PM_H.keras"
modelO_path = "../../models/21PM_O.keras"
results_path = "../../tests/21PM_all_LR"

z_range, rho_H, rho_O = do_slit(9.0, -6.518659136481945, -22.264863831198454, 1.0, -2.0, 8000, modelH_path, modelO_path, results_path)
